In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy 
import re
import string




In [2]:

def cleanText(txt):
    whitespace = string.whitespace
    punctuation = '!"#$%&\'-()*+:;<=>?[\\]^_`{|}~'
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removeWhitespace = text.translate(tableWhitespace)
    removePunctuation = removeWhitespace.translate(tablePunctuation)

    return str(removePunctuation)

In [3]:
model_ner = spacy.load('./output/model-best/')

In [4]:
image = cv2.imread('./data/011.jpeg')
# cv2.imshow('bill',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

tess_data = pytesseract.image_to_data(image)
tess_list = list(map(lambda x: x.split('\t'), tess_data.split('\n')))
df = pd.DataFrame(tess_list[1:], columns= tess_list[0])
df.dropna(inplace=True)
df['text'] =df['text'].apply(cleanText)

df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)

doc = model_ner(content)

invoice no 68688408 date of issue seller torres and sons 4052 brown roads haysstad, nj 06418 tax id 953842239 iban gb84xsno58142814072766 items no. description qty tks nike 747998401 team hustle 3,00 d7 kids blue mid top basketball shoes size 4y us 2. jordan nike kid air 1 retro high 1,00 black gray elephant size 7y 838850013 summary vat 10 total um each each 03/20/2016 client dennis, levy and bowen 523 leslie manors lake colleenville, ct 12053 tax id 904829186 net price net worth vat 10,99 32,97 10 50,00 50,00 10 net worth vat 82,97 8,30 82,97 8,30 gross worth 36,27 55,00 gross worth 91,27 91,27


In [5]:
from spacy import displacy

In [6]:
# displacy.serve(doc,style='ent')

In [7]:
displacy.render(doc,style='ent')

In [8]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [9]:
doc_text = docjson['text']

In [10]:
df_tokens = pd.DataFrame(docjson['tokens'])
df_tokens.head()

,id,start,end
0,0,0,7
1,1,8,10
2,2,11,19
3,3,20,24
4,4,25,27


In [11]:
df_tokens = pd.DataFrame(docjson['tokens'])
df_tokens['token'] = df_tokens[['start','end']].apply(lambda x:doc_text[x['start']:x['end']], axis = 1)

df_tokens.head(10)

,id,start,end,token
0,0,0,7,invoice
1,1,8,10,no
2,2,11,19,68688408
3,3,20,24,date
4,4,25,27,of
5,5,28,33,issue
6,6,34,40,seller
7,7,41,47,torres
8,8,48,51,and
9,9,52,56,sons


In [12]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
df_tokens = pd.merge(df_tokens,right_table, how = "left", on='start')

In [13]:
df_tokens.head(10)

,id,start,end,token,label
0,0,0,7,invoice,NaN
1,1,8,10,no,NaN
2,2,11,19,68688408,B-ID
3,3,20,24,date,NaN
4,4,25,27,of,NaN
5,5,28,33,issue,NaN
6,6,34,40,seller,NaN
7,7,41,47,torres,B-SN
8,8,48,51,and,I-SN
9,9,52,56,sons,I-SN


In [14]:
df_tokens.fillna('O', inplace=True)
df_tokens.head(10)

,id,start,end,token,label
0,0,0,7,invoice,O
1,1,8,10,no,O
2,2,11,19,68688408,B-ID
3,3,20,24,date,O
4,4,25,27,of,O
5,5,28,33,issue,O
6,6,34,40,seller,O
7,7,41,47,torres,B-SN
8,8,48,51,and,I-SN
9,9,52,56,sons,I-SN


In [15]:
df_clean['end']= df_clean['text'].apply(lambda x:len(x)+1).cumsum() - 1
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x['end'] - len(x['text']), axis = 1)

In [16]:
df_clean.head(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start
4,5,1,1,1,1,1,136,77,130,26,96,invoice,7,0
5,5,1,1,1,1,2,283,83,53,20,96,no,10,8
6,5,1,1,1,1,3,353,77,181,26,96,68688408,19,11
10,5,1,2,1,1,1,136,146,69,22,96,date,24,20
11,5,1,2,1,1,2,218,145,28,23,96,of,27,25
12,5,1,2,1,1,3,258,145,83,23,96,issue,33,28
16,5,1,3,1,1,1,135,447,117,25,96,seller,40,34
20,5,1,4,1,1,1,141,511,88,20,96,torres,47,41
21,5,1,4,1,1,2,240,510,49,21,96,and,51,48
22,5,1,4,1,1,3,302,510,63,21,96,sons,56,52


In [17]:
df_info = pd.merge(df_clean,df_tokens[['start','token','label']],how="left",on="start")

In [18]:
df_info.head(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
0,5,1,1,1,1,1,136,77,130,26,96,invoice,7,0,invoice,O
1,5,1,1,1,1,2,283,83,53,20,96,no,10,8,no,O
2,5,1,1,1,1,3,353,77,181,26,96,68688408,19,11,68688408,B-ID
3,5,1,2,1,1,1,136,146,69,22,96,date,24,20,date,O
4,5,1,2,1,1,2,218,145,28,23,96,of,27,25,of,O
5,5,1,2,1,1,3,258,145,83,23,96,issue,33,28,issue,O
6,5,1,3,1,1,1,135,447,117,25,96,seller,40,34,seller,O
7,5,1,4,1,1,1,141,511,88,20,96,torres,47,41,torres,B-SN
8,5,1,4,1,1,2,240,510,49,21,96,and,51,48,and,I-SN
9,5,1,4,1,1,3,302,510,63,21,96,sons,56,52,sons,I-SN


In [19]:
bb_df = df_info.query("label != 'O' ")
img = image.copy()
for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x= int(x)
    y = int(y)
    w = int(w)
    h = int(h)

    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    # cv2.putText(img, str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(0,0,255),2)

cv2.imshow('prediction',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [20]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
2,5,1,1,1,1,3,353,77,181,26,96,68688408,19,11,68688408,ID
7,5,1,4,1,1,1,141,511,88,20,96,torres,47,41,torres,SN
8,5,1,4,1,1,2,240,510,49,21,96,and,51,48,and,SN
9,5,1,4,1,1,3,302,510,63,21,96,sons,56,52,sons,SN
20,5,1,5,1,2,2,228,688,400,21,89,gb84xsno58142814072766,137,115,gb84xsno58142814072766,IBAN
63,5,1,12,1,1,1,805,145,172,26,96,03/20/2016,372,362,03/20/2016,DATE
65,5,1,14,1,1,1,837,510,100,24,95,"dennis,",387,380,dennis,CN
66,5,1,14,1,1,2,951,511,61,26,96,levy,392,388,levy,CN
67,5,1,14,1,1,3,1024,510,48,21,96,and,396,393,and,CN
68,5,1,14,1,1,4,1086,511,89,20,96,bowen,402,397,bowen,CN


In [21]:
class group_gen():
    def __init__(self):
        self.id = 0
        self.text = ''

    def get_group(self,text):
        if self.text == text:
            return self.id
        else:
            self.id += 1
            self.text = text
            return self.id


grp_gen = group_gen()

In [22]:
bb_df['group'] = bb_df['label'].apply(grp_gen.get_group)

In [23]:
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

In [24]:
col_group = ['left','top','right','bottom','label','token', 'group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [25]:
img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':lambda x: x.iloc[0],
    'token':lambda x: " ".join(x)
        
})

In [26]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,353,534,77,103,ID,68688408
2,141,365,510,531,SN,torres and sons
3,228,628,688,709,IBAN,gb84xsno58142814072766
4,805,977,145,171,DATE,03/20/2016
5,837,1175,510,537,CN,dennis levy and bowen
6,1435,1502,1402,1422,TOTAL,"91,27"


In [27]:
img_bb = img.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb,(l,t),(r,b),(255,0,0),2)
    cv2.putText(img_bb,label,(l,t),cv2.FONT_HERSHEY_PLAIN,1,(0,0,255),2)

cv2.imshow('Bounding Boxes',img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [28]:
def parser(text, label): 
    if label == 'ID' or label == 'TOTAL':
        text = text.lower()
        text = re.sub(r'\D', '', text)
        
    elif label == 'DATE' :
        text = text.lower()
        allowed_special_characters = '/'
        text = re.sub(r'[^0-9{} ]'.format(allowed_special_characters), '', text)

    elif label == 'SN' or label == 'CN':
        text = text.lower()
        allowed_special_characters = ','
        text = re.sub(r'[^A-Za-z{} ]'.format(allowed_special_characters), '', text)

    elif label == 'IBAN' :
        text = text.lower()
        text = re.sub(r'[^A-Za-z0-9{} ]','', text)
        
    return text
    

In [29]:
info_array = df_info[['token','label']].values
entities = dict(ID=[],DATE=[],SN=[],CN=[],IBAN=[],TOTAL=[])
prvs = 'O'
for token, label in info_array:
    bio_tag = label[:1]
    label_tag = label[2:]

    text = parser(token, label_tag)
    if bio_tag in ('B', 'I'):
        if prvs != label_tag:
            entities[label_tag].append(text)
        else:
            if bio_tag == 'B':
                entities[label_tag].append(text)

            else:
                if label_tag in ('SN','CN'):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
                else:
                    entities[label_tag][-1] = entities[label_tag][-1]  + text

    prvs = label_tag
        

In [30]:
entities

{'ID': ['68688408'],
 'DATE': ['03/20/2016'],
 'SN': ['torres and sons'],
 'CN': ['dennis levy and bowen'],
 'IBAN': ['gb84xsno58142814072766'],
 'TOTAL': ['9127']}